# Pre-compute Global EII

1. Build global computation for each component
2. Combine into single EII image
3. Export with global region - GEE tiles internally

**Prerequisites:**
- Natural NPP tiles at `natural_npp` asset folder
- Observed NPP annual tiles at CLMS asset folder

In [ ]:
import datetime

import ee

ee.Initialize()

from eii.compute.compositional import calculate_compositional_integrity
from eii.compute.integrity import combine_components
from eii.compute.npp import calculate_functional_integrity
from eii.compute.settings import (
    DEFAULT_AGGREGATION_METHOD,
    MAGNITUDE_WEIGHT,
    OBSERVED_NPP_ASSET_PATH,
    OBSERVED_NPP_YEAR_RANGE,
    SEASONALITY_WEIGHT,
    SPATIAL_RESOLUTION,
)
from eii.utils.gee import create_assets_folder

OUTPUT_ASSET_PATH = "projects/landler-open-data/assets/eii/global/eii_global_v1"

RESOLUTION = SPATIAL_RESOLUTION
AGGREGATION_METHOD = DEFAULT_AGGREGATION_METHOD

NATURAL_NPP_ONESHOT_PATH = (
    "projects/landler-open-data/assets/eii/intermediate/functional/predictions/natural_npp_v1"
)

# Global bounds (excluding polar regions where data is sparse)
GLOBAL_BOUNDS = ee.Geometry.Rectangle([-180, -60, 180, 75], proj="EPSG:4326", geodesic=False)


# GLOBAL_BOUNDS = ee.Geometry.Rectangle([10.5, 47.5, 12.5, 49.0], proj="EPSG:4326", geodesic=False)
# OUTPUT_ASSET_PATH = "projects/landler-open-data/assets/eii/global/eii_test"


print(f"Output: {OUTPUT_ASSET_PATH}")
print(f"Resolution: {RESOLUTION}m")
print(f"Method: {AGGREGATION_METHOD}")
print(f"Observed NPP range: {OBSERVED_NPP_YEAR_RANGE}")

Output: projects/landler-open-data/assets/eii/global/eii_global_v1
Resolution: 300m
Method: min_fuzzy_logic
Observed NPP range: ['2022-01-01', '2025-01-01']
Percentiles loaded: ['p5', 'p10', 'p15', 'p20', 'p25', 'p30', 'p35', 'p40', 'p45', 'p50', 'p55', 'p60', 'p65', 'p70', 'p75', 'p80', 'p85', 'p90', 'p95']


## Functional Integrity

In [ ]:
functional_result = calculate_functional_integrity(
    aoi=None,
    year_range=OBSERVED_NPP_YEAR_RANGE,
    include_seasonality=True,
    natural_npp_asset_path=NATURAL_NPP_ONESHOT_PATH,
    observed_npp_asset_path=OBSERVED_NPP_ASSET_PATH,
    absolute_diff_percentile="p80",
)

functional_integrity = functional_result["functional_integrity"].toFloat()
relative_deviation = functional_result["relative_npp"].subtract(1).abs()

npp_debug_images = [
    functional_result["actual_npp"],
    functional_result["relative_npp"],
    relative_deviation.rename("relative_deviation"),
    functional_result["proportional_score"],
    functional_result["npp_difference"],
    functional_result["absolute_score"],
    functional_result["magnitude_integrity"],
]

npp_debug_names = [
    "actual_npp",
    "relative_npp",
    "relative_deviation",
    "proportional_score",
    "npp_difference",
    "npp_difference_penalty",
    "magnitude_integrity",
]

if functional_result["seasonality_integrity"] is not None:
    npp_debug_images.append(functional_result["seasonality_integrity"])
    npp_debug_names.append("seasonality_integrity")

npp_debug = ee.Image.cat(npp_debug_images).rename(npp_debug_names).toFloat()

In [29]:
task = ee.batch.Export.image.toAsset(
    image=npp_debug,
    description="debug",
    assetId="projects/landler-open-data/assets/functional_debug",
    region=GLOBAL_BOUNDS,
    scale=RESOLUTION,
    crs="EPSG:4326",
    maxPixels=1e13,
    pyramidingPolicy={".default": "mean"},
)
task.start()

## Structural Integrity

In [3]:
from eii.compute.structural import calculate_structural_integrity

structural_integrity = calculate_structural_integrity(aoi=None).toFloat()

## Compositional Integrity

In [ ]:
compositional_integrity = calculate_compositional_integrity(
    aoi=None,
    year=2020,
).toFloat()

## Combine Components into EII

In [ ]:
eii = combine_components(
    functional=functional_integrity,
    structural=structural_integrity,
    compositional=compositional_integrity,
    method=AGGREGATION_METHOD,
).toFloat()

eii_multiband = ee.Image.cat(
    [
        functional_integrity,
        structural_integrity,
        compositional_integrity,
        eii,
    ]
).set(
    {
        "aggregation_method": AGGREGATION_METHOD,
        "resolution_m": RESOLUTION,
        "observed_npp_start": OBSERVED_NPP_YEAR_RANGE[0],
        "observed_npp_end": OBSERVED_NPP_YEAR_RANGE[1],
        "created_date": datetime.datetime.now().strftime("%Y-%m-%d"),
        "magnitude_weight": MAGNITUDE_WEIGHT,
        "seasonality_weight": SEASONALITY_WEIGHT,
    }
)

## Export

In [6]:
parent_folder = "/".join(OUTPUT_ASSET_PATH.split("/")[:-1])
create_assets_folder(parent_folder)

try:
    existing = ee.data.getAsset(OUTPUT_ASSET_PATH)
    print(f"Asset already exists: {OUTPUT_ASSET_PATH}")
    RUN_EXPORT = False
except ee.EEException:
    task = ee.batch.Export.image.toAsset(
        image=eii_multiband,
        description="EII_Global_OneShot",
        assetId=OUTPUT_ASSET_PATH,
        region=GLOBAL_BOUNDS,
        scale=RESOLUTION,
        crs="EPSG:4326",
        maxPixels=1e13,
        pyramidingPolicy={".default": "mean"},
    )
    task.start()

In [10]:
task.status()

{'state': 'RUNNING',
 'description': 'EII_Global_OneShot',
 'priority': 100,
 'creation_timestamp_ms': 1768521998532,
 'update_timestamp_ms': 1768522222212,
 'start_timestamp_ms': 1768522007974,
 'task_type': 'EXPORT_IMAGE',
 'attempt': 1,
 'id': 'DY2CL3YQQUUTUVIMANNYS4YO',
 'name': 'projects/science-428407/operations/DY2CL3YQQUUTUVIMANNYS4YO'}